In [1]:
%gui qt

In [2]:
from importlib import reload
from pprint import pprint

import numpy as np

import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart, Node as pgNode
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib

from plottr.data import datadict; reload(datadict)
from plottr.data.datadict import togrid, DataDict, GridDataDict

In [22]:
def testdata_1d(nvals=11):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx=3, ny=3, nz=3):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1), unit='mX'),
        y = dict(values=yy.reshape(-1), unit='kY'),
        z = dict(values=zz.reshape(-1), unit='MZ'),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z'], unit='OMFG'),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z'], unit='WTF'),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'], unit='FML')
    )
    return d

In [23]:
from plottr import node; reload(node)
from plottr.node import Node, DataSelector

class DataSelectorWidget(QtGui.QWidget):
    
    newDataStructure = QtCore.pyqtSignal()
    axesChoiceUpdated = QtCore.pyqtSignal(str, str)
    dataSelected = QtCore.pyqtSignal()
    
    def __init__(self, parent=None, **kw):
        super().__init__(parent=parent, **kw)
        
        # Axes selection
        self._axesOptions = []
        self.xCombo = QtGui.QComboBox()
        self.yCombo = QtGui.QComboBox()
        self.xCombo.currentTextChanged.connect(
            lambda choice: self.selectAx('x', choice))
        self.yCombo.currentTextChanged.connect(
            lambda choice: self.selectAx('y', choice))
        
        axLayout = QtGui.QFormLayout()
        axLayout.addRow('x-axis', self.xCombo)
        axLayout.addRow('y-axis', self.yCombo)
        axGroup = QtGui.QGroupBox('Axis selection')
        axGroup.setLayout(axLayout)
        
        # Data fields
        self._triggerUpdateSelectables = True
        self._dataOptions = {}
        self._selectedData = []
        
        self.dataLayout = QtGui.QFormLayout()
        dataGroup = QtGui.QGroupBox('Data selection')
        dataGroup.setLayout(self.dataLayout)
        
        mainLayout = QtGui.QVBoxLayout(self)
        mainLayout.addWidget(axGroup)
        mainLayout.addWidget(dataGroup)
        
        # TODO this should just be a RESET method or so.
#         self.newDataStructure.connect(self.updateAxOptions)
#         self.newDataStructure.connect(self.updateDataOptions)
#         self.newDataStructure.connect(self.updateSelectableData)
        self.axesChoiceUpdated.connect(self.updateSelectableData)
        

    @QtCore.pyqtSlot(object)
    def setDataStructure(self, newStructure):
        print('update structure:')
        pprint(newStructure)
        self.dataStructure = newStructure
        axes = newStructure.axes_list()
        if axes != self._axesOptions:
            self._axesOptions = axes
            self.newDataStructure.emit()
    
    @QtCore.pyqtSlot()
    def resetDataStructure(self)
        
        
    @QtCore.pyqtSlot()
    def updateAxOptions(self):
        print('update ax opts')
        axOptions = [''] + self._axesOptions
        for combo in self.xCombo, self.yCombo:
            combo.clear()
            combo.addItems(axOptions)
        
    def _deleteDataOptions(self, names):
        for name in names:
            v = self._dataOptions[name]
            self.dataLayout.removeWidget(v['widget'])
            self.dataLayout.removeWidget(v['label'])
            v['widget'].deleteLater()
            v['label'].deleteLater()
            del self._dataOptions[name]
    
    @QtCore.pyqtSlot()
    def updateDataOptions(self):
        print('update data opts')
        
        delete = []
        for k, v in self._dataOptions.items():
            delete.append(k)
        self._deleteDataOptions(delete)
        
        for n in self.dataStructure.dependents():
            if n not in self._dataOptions:               
                lbl = QtGui.QLabel(n)
                chk = QtGui.QCheckBox()
                chk.stateChanged.connect(lambda x: self.updateSelectableData())
                self._dataOptions[n] = dict(label=lbl, widget=chk)
                self.dataLayout.addRow(lbl, chk)
        
    def updateSelectableData(self, *arg):
        print('update selectable data')
        
        if self._triggerUpdateSelectables:
            self._triggerUpdateSelectables = False
            
            curx = self.xCombo.currentText()
            cury = self.yCombo.currentText()
            curdata = []
            curaxes = []
            for k, v in self._dataOptions.items():
                if v['widget'].isChecked():
                    curdata.append(k)
            
            if len(curdata) > 0:
                curaxes = self.dataStructure[curdata[0]]['axes']
                
            for k, v in self._dataOptions.items():
                
                if curx == '':
                    v['widget'].setDisabled(True)
                    v['widget'].setChecked(False)
                    
                elif curx not in self.dataStructure[k]['axes']:
                    v['widget'].setDisabled(True)
                    v['widget'].setChecked(False)
                    
                elif (len(curaxes) > 0) and (self.dataStructure[k]['axes'] != curaxes):
                    v['widget'].setDisabled(True)
                    v['widget'].setChecked(False)

                else:
                    v['widget'].setEnabled(True)                    
            
            self._triggerUpdateSelectables = True
        
    
    def selectAx(self, dimName, axName):
        print('select ax', dimName, axName)
        
        emit = True
        
        if dimName == 'x':
            if axName != '' and axName == self.yCombo.currentText():
                emit = False
                self.yCombo.setCurrentText('')
        elif dimName == 'y':
            if axName != '' and axName == self.xCombo.currentText():
                emit = False
                self.xCombo.setCurrentText('')
                
        order = {}
        curx = self.xCombo.currentText()
        cury = self.yCombo.currentText()
        if curx != '':
            order[curx] = 0
        if cury != '':
            order[cury] = 1
        
        if emit:
            self.axesChoiceUpdated.emit()
            print('New axes choices: {}, {}'.format(curx, cury))

        
class DSelNode(DataSelector):
    
    updated = QtCore.pyqtSignal()
    sendDataStructure = QtCore.pyqtSignal(object)
    useUi = True
    
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)
        
        if self.useUi:
            self.ui = DataSelectorWidget()
            self.sendDataStructure.connect(self.ui.setDataStructure)
        else:
            self.ui = None
            
            
    def process(self, **kw):
        data = kw['dataIn']
        self.sendDataStructure.emit(data.structure())
        return super().process(**kw)
    
        
    def update(self, signal=True):
        super().update(signal=signal)
        self.updated.emit()
    
    def ctrlWidget(self):            
        return self.ui

In [24]:
Node.raiseExceptions = True
    
nodelib = fclib.NodeLibrary()
nodelib.addNodeType(Node, [('Basic')])
nodelib.addNodeType(DSelNode, [('Basic')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib
fc.setInput(dataIn=testdata_3d())

selector = fc.createNode('DataSelector')

fc.connectTerminals(fc['dataIn'], selector['dataIn'])
fc.connectTerminals(selector['dataOut'], fc['dataOut'])

dialog = QtGui.QDialog()
layout = QtGui.QVBoxLayout(dialog)
layout.addWidget(selector.ctrlWidget())
dialog.show()

update structure:
{'data': {'axes': ['x', 'y', 'z'], 'info': {'shape': (27,)}, 'unit': 'OMFG'},
 'different_data': {'axes': ['z', 'y', 'x'],
                    'info': {'shape': (27,)},
                    'unit': 'FML'},
 'more_data': {'axes': ['x', 'y', 'z'],
               'info': {'shape': (27,)},
               'unit': 'WTF'},
 'x': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'mX'},
 'y': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'kY'},
 'z': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'MZ'}}
update ax opts
select ax x 
update selectable data
New axes choices: , 
select ax y 
update selectable data
New axes choices: , 
update data opts
update selectable data


In [25]:
fc.inputValues(), fc.output()

({'dataIn': None}, {'dataOut': {}})

In [26]:
selector.signalUpdate = True
selector.selectedData = ['data', 'more_data']

update structure:
{'data': {'axes': ['x', 'y', 'z'], 'info': {'shape': (27,)}, 'unit': 'OMFG'},
 'different_data': {'axes': ['z', 'y', 'x'],
                    'info': {'shape': (27,)},
                    'unit': 'FML'},
 'more_data': {'axes': ['x', 'y', 'z'],
               'info': {'shape': (27,)},
               'unit': 'WTF'},
 'x': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'mX'},
 'y': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'kY'},
 'z': {'axes': [], 'info': {'shape': (27,)}, 'unit': 'MZ'}}


In [27]:
fc.inputValues(), fc.output()

({'dataIn': None},
 {'dataOut': {'x': {'values': array([  0.,   5.,  10.]),
    'unit': 'mX',
    'axes': [],
    'info': {}},
   'y': {'values': array([-5.,  0.,  5.]),
    'unit': 'kY',
    'axes': [],
    'info': {}},
   'z': {'values': array([0, 1, 2], dtype=int64),
    'unit': 'MZ',
    'axes': [],
    'info': {}},
   'data': {'values': array([[[ 1.65324113,  1.69623546,  1.42867135],
            [ 0.84879431,  0.44077046,  0.54911932],
            [-0.03030049, -0.44743718, -0.92221908]],
    
           [[ 0.75491088,  0.40676904,  0.84942534],
            [ 0.87980277,  0.84373585,  0.49444572],
            [ 0.44768172, -0.24285784,  0.38632333]],
    
           [[ 0.07981546, -0.48869775, -0.42246085],
            [ 0.21648197,  0.67296697,  0.08418625],
            [ 1.01086088,  1.65047597,  0.91909626]]]),
    'axes': ['x', 'y', 'z'],
    'unit': 'OMFG',
    'info': {}},
   'more_data': {'values': array([[[ 0.95906053,  0.43826484,  0.79822944],
            [ 0.88447516, 

In [ ]:
class Test(dict):
    
    def metaItems(self):
        meta = {}
        for k, v in self.items():
            if isinstance(k, str) and len(k) > 3 and k[:2] == '__' and k[-2:] == '__':
                yield k, v

In [ ]:
t = Test()
t['abc'] = 123
t['def'] = 456
t['__abc__'] = '123'
t['__def_'] = '456'

In [ ]:
for k, v in t.items():
    print(k, v)

In [ ]:
for k, v in t.metaItems():
    print(k, v)